<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Найти-аэропорт-с-минимальной-задержкой-вылета" data-toc-modified-id="Найти-аэропорт-с-минимальной-задержкой-вылета-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Найти аэропорт с минимальной задержкой вылета</a></span></li><li><span><a href="#Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport" data-toc-modified-id="Самая-пунктуальная-авиакомпания-на-прилет-в-Los-Angeles-International-Airport-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport</a></span></li><li><span><a href="#Аэропорт-с-самой-большой-долей-руления-на-1-самолетовылет" data-toc-modified-id="Аэропорт-с-самой-большой-долей-руления-на-1-самолетовылет-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Аэропорт с самой большой долей руления на 1 самолетовылет</a></span></li><li><span><a href="#Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета" data-toc-modified-id="Построить-модель-которая-будет-выбирать-топ-3-аэропорта-прилета-(вероятность-опоздания-минимальная-–-RMSE-метрика),-в-зависимости-от-аэропорта-вылета-4"><span class="toc-item-num">4&nbsp;&nbsp;</span>Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета</a></span><ul class="toc-item"><li><span><a href="#Подготовка-данных" data-toc-modified-id="Подготовка-данных-4.1"><span class="toc-item-num">4.1&nbsp;&nbsp;</span>Подготовка данных</a></span></li><li><span><a href="#Выбор-параметров-и-получение-результата" data-toc-modified-id="Выбор-параметров-и-получение-результата-4.2"><span class="toc-item-num">4.2&nbsp;&nbsp;</span>Выбор параметров и получение результата</a></span></li></ul></li></ul></div>

[Данные - https://www.kaggle.com/usdot/flight-delays  ](https://www.kaggle.com/usdot/flight-delays) 

Описание - Сводная информация о количестве своевременных, задержанных, отмененных и измененных рейсов в США за 2015 год (5.8 млн записей)
Задачи:
1	Найти аэропорт с минимальной задержкой вылета 
2	Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport
3	Аэропорт с самой большой долей руления на 1 самолетовылет
4	Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика),  в зависимости от аэропорта вылета  

In [1]:
#%%time
import pandas as pd
import datetime as dt
import numpy as np
import random
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error
from math import sqrt
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression


In [ ]:
airlines= pd.read_csv('data/airlines.csv')
airports= pd.read_csv('data/airports.csv')
flights= pd.read_csv('data/flights.csv',dtype={'ORIGIN_AIRPORT':str,'DESTINATION_AIRPORT':str})

In [ ]:
flights1=flights.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

In [ ]:
flights1=flights1.merge(airports[['IATA_CODE','AIRPORT']], how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE',], axis = 1)

In [ ]:
flights1=flights1.merge(airlines, how='inner', left_on='AIRLINE', right_on='IATA_CODE').drop (['IATA_CODE'], axis = 1)

In [ ]:
#flights_real=flights1.query("DIVERTED==0 & CANCELLED==0")

In [ ]:
flights_real= flights1[~flights1.DIVERTED.isin([1])& ~flights1.CANCELLED.isin([1])].copy()

In [ ]:
flights_real.info()

In [ ]:
flights_real.nunique()

# 	Найти аэропорт с минимальной задержкой вылета

In [ ]:
flights_dep_delay=flights_real.query("DEPARTURE_DELAY>=0").groupby(['ORIGIN_AIRPORT','AIRPORT_x' ])['DEPARTURE_DELAY'].median().sort_values()
flights_dep_delay.head(1)

In [ ]:
flights_real.query("DEPARTURE_DELAY>=0").boxplot('DEPARTURE_DELAY',figsize = (14, 14))

Использована медиана, а не среднее, так как большое количество выбросов по полю DEPARTURE_DELAY.

#  Самая пунктуальная авиакомпания на прилет в Los Angeles International Airport 

In [ ]:
flights_ariv_delay=flights_real.query("ARRIVAL_DELAY>=0 & DESTINATION_AIRPORT=='LAX'").groupby(['AIRLINE_x','AIRLINE_y' ])['ARRIVAL_DELAY'].median().sort_values()
flights_ariv_delay.head(1)

In [ ]:
flights_real.query("ARRIVAL_DELAY>=0 & DESTINATION_AIRPORT=='LAX'").boxplot('ARRIVAL_DELAY',figsize = (14, 14))

Использована медиана, а не среднее, так как большое количество выбросов по полю ARRIVAL_DELAY.

# Аэропорт с самой большой долей руления на 1 самолетовылет

In [ ]:
flights_TAXI_OUT= (flights_real.groupby(['ORIGIN_AIRPORT','AIRPORT_x']).TAXI_OUT.sum()/flights_real.groupby(['ORIGIN_AIRPORT','AIRPORT_x']).YEAR.count()).sort_values(ascending=False)
flights_TAXI_OUT.head(1)

# Построить модель которая будет выбирать топ 3 аэропорта прилета (вероятность опоздания минимальная – RMSE метрика), в зависимости от аэропорта вылета

## Подготовка данных 

In [ ]:
def make_date(df):
 return dt.date(df.YEAR,df.MONTH,df.DAY)

In [ ]:
flights_real['dDate']=flights_real.apply(make_date, axis=1)
flights_real['dDate']=pd.to_datetime(flights_real['dDate'], errors='coerce')

In [ ]:
df = flights_real[['dDate','ORIGIN_AIRPORT','DESTINATION_AIRPORT','ARRIVAL_DELAY']]

In [ ]:
df=df.query("ARRIVAL_DELAY>=0")

In [ ]:
airports['num']=airports.index

In [ ]:
df=df.merge(airports[['num','IATA_CODE']], how='inner', left_on='ORIGIN_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE','ORIGIN_AIRPORT'], axis = 1)

In [ ]:
df.rename(columns={'num':'ORIGIN_AIRPORT_num'}, inplace=True)

In [ ]:
df=df.merge(airports[['num','IATA_CODE']], how='inner', left_on='DESTINATION_AIRPORT', right_on='IATA_CODE').drop (['IATA_CODE','DESTINATION_AIRPORT',], axis = 1)

In [ ]:
df.rename(columns={'num':'DESTINATION_AIRPORT_num'}, inplace=True)

In [ ]:
def make_features(data, max_lag, rolling_mean_size):
    data['YEAR'] =data.dDate.dt.year
    data['MONTH'] =data.dDate.dt.month
    data['DAY'] =data.dDate.dt.day
    data['DAYOFWEEK'] =data.dDate.dt.dayofweek
    data['is_weekend'] = data.DAYOFWEEK.isin([5,6])*1
    for lag in range (1, max_lag+1):
        data['ARRIVAL_DELAY_{}'.format(lag)] = data['ARRIVAL_DELAY'].shift(lag)  
    data['y_mean'] = data['ARRIVAL_DELAY'].shift().rolling(rolling_mean_size).mean().copy()     

In [ ]:
make_features(df,7,3)
df.dropna(inplace=True)

In [ ]:
df = df.set_index(['ORIGIN_AIRPORT_num','DESTINATION_AIRPORT_num','dDate'])

In [ ]:
df

In [ ]:
def predict_date_today(a):
    df_pred=a[:1].copy()
    df_pred=df_pred.reset_index()
    df_pred['dDate']=dt.date.today()
    df_pred['dDate']=pd.to_datetime(df_pred['dDate'], errors='coerce')
    make_features(df_pred,7,3)
    df_pred=df_pred.fillna(0).drop(['ARRIVAL_DELAY','DESTINATION_AIRPORT_num','ORIGIN_AIRPORT_num'],axis = 1)
    df_pred = df_pred.set_index(['dDate'])
    return df_pred
p=predict_date_today(df)

In [ ]:
def make_predict(a,b):
    features = a.drop(['ARRIVAL_DELAY'],axis = 1)
    target = a['ARRIVAL_DELAY']
    X_train,X_test,y_train, y_test = train_test_split(features,target, shuffle=False, test_size=0.1)
    model_lr = LinearRegression()
    model_lr.fit(X_train,y_train)
    y_predicted_lr = model_lr.predict(X_test)
    return round(np.sqrt(mean_squared_error(y_test, y_predicted_lr)),1),round(model_lr.predict(b)[0],1)

## Выбор параметров и получение результата

In [ ]:
x=random.randrange (0, 321)
df_random=df.xs(x)
print('Случайно выбранный аэропорт вылета ORIGIN_AIRPORT-', airports[airports.num==x][['IATA_CODE', 'AIRPORT','CITY']])

In [ ]:
#Проверка на аэропорты назанчения с единичным набором данных по дате, удаляем такие
df_random['Col3'] = df_random.groupby(level=0)['YEAR'].transform(sum)
df_random = df_random[~df_random.Col3.isin([2015])].copy()
df_random.drop (['Col3'], axis = 1, inplace=True)

In [ ]:
df_DESTINATION_AIRPORT=pd.DataFrame(df_random.groupby(level=0)['ARRIVAL_DELAY'].mean())
df_DESTINATION_AIRPORT['count']=pd.DataFrame(df_random.groupby(level=0)['ARRIVAL_DELAY'].count())
df_DESTINATION_AIRPORT.sort_values(by='ARRIVAL_DELAY')

In [ ]:
test = []
for i in (df_DESTINATION_AIRPORT.index.get_level_values(0)):
        test.append([i,make_predict(df_random.xs(i),p)[0],make_predict (df_random.xs(i),p)[1],
                     make_predict(df_random.xs(i),p)[0]+make_predict (df_random.xs(i),p)[1]])
test = pd.DataFrame(test)
test.columns = ['DESTINATION_AIRPORT_num','rmse','delay_predict','result'] 
test=test.merge(airports, how='inner', left_on='DESTINATION_AIRPORT_num', right_on='num').drop (['num'], axis = 1)
test=test.set_index('DESTINATION_AIRPORT_num')
test.sort_values(by='result').head(3)